# Homework 1

### DUE: 02/22/2017 before class at 9:30am

This homework is for practicing Python’s *generators* and the design of *streaming* algorithms in general. We’re going to use the **citibike.csv** data set that we had played around with in the labs, and another Citibike's data set called **citibike_docking_events.csv**. Both of them are available CUNY Blackboard under *Data Sets* section. You are required to turn in this notebook with all the parts filled in place of <###>. Your notebook must be named BDM\_HW1.ipynb.

## Task 1 (3 points)

Your task is to write a generator to extract the first ride of the day from a Citibike data stream. The data stream is sorted based on starting times. The first ride of the day is interpreted as the ride with the earliest starting time of a day. For the sample data, which is a week worth of citibike records, your generator should only generate 7 items (one for each day).

You are given a template with the sample generator **firstRide**. The generator currently takes in **csv.DictReader** generator and output its first element. Please adjust this generator to output the first ride of the day for the entire stream as specified above. The output of the generator must be in the same format as csv.DictReader. You can think of this generator as a filter only passing certain records out.

In [50]:
import csv
import datetime 

### NOTE: You need to change the body of the generator firstRide
### in order to output trip record that appeared first in each day
### using the same dict format as csv.DictReader.

def firstRide(reader):
    #<YOUR CODE HERE>
    firstBike = []
    weekdays = [1,2,3,4,5,6,7,8]
    day  = 0
    for row in reader:
        datetimeObject = datetime.datetime.strptime(row["starttime"], '%Y-%m-%d %H:%M:%S+%f')
        if (datetimeObject.day == weekdays[day]):
            firstBike.append(row) #str(datetimeObject)
            day+=1
    
    return (firstBike)
   
    
### NOTE: You SHOULD NOT modify the code below. If you
### write your firstRide generator above correctly, the
### code below will output the correct information

with open('citibike.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    for row in firstRide(reader):
        print ','.join(map(row.get, reader.fieldnames))


1,,801,2015-02-01 00:00:00+00,2015-02-01 00:14:00+00,521,8 Ave & W 31 St,40.75044999,-73.99481051,423,W 54 St & 9 Ave,40.76584941,-73.98690506,17131,Subscriber,1978,2
6442,,199,2015-02-02 00:02:00+00,2015-02-02 00:05:00+00,442,W 27 St & 7 Ave,40.746647,-73.993915,489,10 Ave & W 28 St,40.75066386,-74.00176802,20684,Subscriber,1992,1
7901,,704,2015-02-03 00:00:00+00,2015-02-03 00:12:00+00,387,Centre St & Chambers St,40.71273266,-74.0046073,2008,Little West St & 1 Pl,40.70569254,-74.01677685,20328,Subscriber,1982,1
12655,,146,2015-02-04 00:00:00+00,2015-02-04 00:02:00+00,237,E 11 St & 2 Ave,40.73047309,-73.98672378,438,St Marks Pl & 1 Ave,40.72779126,-73.98564945,15253,Subscriber,1969,1
21628,,1034,2015-02-05 00:00:00+00,2015-02-05 00:17:00+00,497,E 17 St & Broadway,40.73704984,-73.99009296,461,E 20 St & 2 Ave,40.73587678,-73.98205027,20290,Subscriber,1971,1
30836,,212,2015-02-06 00:01:00+00,2015-02-06 00:05:00+00,491,E 24 St & Park Ave S,40.74096374,-73.98602213,472,E 32 St & Park Ave,40

## Task 2 (3 points)

Your task is to **compute the maximum number of active "citibikers"** that were using the Citibike service at any point in time. This the same as computing the maximum number of citibikes that were checked out at a particular time. The input data set is **citibike_docking_events.csv**, which logged all docking and undocking events at all Citibike stations. The description of the fields in this file is as follows:

|Column name|Description|
|--|--|
|time |The timestamp of the event. All events are sorted increasingly by their timestamps. |
|bikeid |The unique ID of the bike involved in this event. |
|station_id |The station ID, where the event happened. |
|event |A string of either *"dock"* or *"undock"* for describing the drop-off or pick-up event, respectively. |

For example, let's assume that on *Feb-01-2015*, there was a user that picked a bike at the station ID *521* at midnight and dropped it at the station ID *423* at 14 minutes past midnight. If the bike that this customer used has the ID of *17131*, then you should see two events being logged in this data set as:

<pre>
...
2015-02-01 00:00:00+00,17131,521,undock
...
2015-02-01 00:14:00+00,17131,423,dock
...
</pre>

You are given the above data set in a streaming fashion (reading in row by row), and must design a streaming algorithm that uses the least possible additional memory to compute the maximum number of active users of the Citibike service. Please modify the code snippet below to complete this task. Your code should only output a single number, which is the number of active users. Of course, you can add additional initialization codes outside of the for loop as needed.
Additional, please provide a brief rationale and/or justification for your design after the code.

In [2]:
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row
            
oldMax = 0 
maxActiveUsers = 0

for row in csvRows('citibike_docking_events.csv'):
    #<YOUR CODE: STREAMING>
    if row["event"] == "undock":
        oldMax += 1
    if row["event"] == "dock":
        oldMax-=1
    if oldMax>=maxActiveUsers:
        maxActiveUsers = oldMax
        
print maxActiveUsers

250


#### RATIONALE AND JUSTIFICATION

*Please explain your solution here*

A generator calculates the values on the fly and forgets them. Generators are especially useful for memory-intensive tasks, where there is no need to keep all of the elements of a memory-heavy list accessible at the same time. In this problem I introduced a new variable "oldMax". oldMax holds the number of current maximum active users at any point during the for loop. oldMax updates maxActiveUsers every time it is greater than or equal to maxActiveUser.  


## Task 3 (4 points)

The objective of this task is identical to Task 2's but you are asked to use the **cibibike.csv** data set instead of the docking events. The main difference (and challenge) is that both pick-up and drop-off event for each trip is now presented as a single record, thus, the drop-off events are not sorted by their timestamps. You are again asked to do this in a streaming fashion that needs to minimize the amount of memory usage. Please modify the code below accordingly, and also with a brief explaination of the solution.

In [1]:
import csv
import datetime

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row
                
maxActiveUsers = 0
oldMax = 0
times = [] 

for row in csvRows('citibike.csv'):
    #<YOUR STREAMING CODE HERE>
    oldMax +=1 #once we arrive at a new row oldmax increases by 1 default
    startTime = datetime.datetime.strptime(row["starttime"], '%Y-%m-%d %H:%M:%S+%f')
    stopTime = datetime.datetime.strptime(row["stoptime"], '%Y-%m-%d %H:%M:%S+%f')
    tempTime = stopTime
    
    times.append(stopTime) #Times holds the stopTime of every row
    
    for objects in times: #This loop finds whether there is a match (or if startTime is greater) and . . .
        if startTime>=objects:
            times.remove(objects) #. . . removes it from the list
            oldMax -= 1 #updates the max once a match is found

    if oldMax >= maxActiveUsers: #update maximum number of users if oldmax is greater than maxActiveUsers 
        maxActiveUsers=oldMax 
    
print maxActiveUsers


250


#### RATIONALE AND JUSTIFICATION

*Please explain your solution here*

Since every row has a 'starttime' and 'stoptime' column it becomes a tedious task to find the maxActiveUsers for any point in time. My approach for this problem is to simly create a list called 'times'. As we stream through every row I simply add the row's 'stoptime' to the list 'times'. As we move down through the data -- if at any point the row's 'starttime' is equal to any of the elements in the list 'times' then that tells me that I have reached the maximum of active users thus far. The 'times' is never larger than 250 elements and iteration through that manny elements is not a time consuming task. 